In [5]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/ukrsyllab.py /content/ukrsyllab.py 
import pandas, re, random
!pip install git+https://github.com/Desklop/Uk_Stemmer
from uk_stemmer import UkStemmer #stemmer for ukrainian 
from ast import literal_eval 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Desklop/Uk_Stemmer to /tmp/pip-req-build-t_mv6rmw
  Running command git clone --filter=blob:none --quiet https://github.com/Desklop/Uk_Stemmer /tmp/pip-req-build-t_mv6rmw
  Resolved https://github.com/Desklop/Uk_Stemmer to commit a700ae1bd9b69ad84d311d089e8bc95ab7fab44d
  Preparing metadata (setup.py) ... done


In [6]:
import ukrsyllab #syllabificator -- initially for russian(https://github.com/Koziev/rusyllab), slightly modified for ukrainain

In [22]:
class Pseudoword_genUK():

  def __init__(self, filename, n_words=300, n_sent=6):
    file = open(filename, "r", encoding="utf8")
    words = file.read().split("\n")

    self.stems = []
    for w in words[:50000]: 
      w = re.sub("\w'\w", "", w) #remove apostrophe words
      w = re.sub("\w-\w", "", w) #remove hyphenated words
      stemmer = UkStemmer()
      w = stemmer.stem_word(w) #stem the words from the dataset
      if w in words and len(w) > 1 and w.lower() == w: 
        if w not in self.stems:
          self.stems.append(w) #append the stems list with unique lower-case two(or more)-syllable stems 

    self.n_words = n_words
    self.n_sent = n_sent

In [8]:
  def probabilities(self): #create a probability matrix for word generation
    sound_prob = {} 
    in_m = {}
    for word in self.stems:
      i = 0 #index value
      morphemes = ukrsyllab.split_words(word.split()) #get syllables -- bigrams used to form a matrix
      if len(morphemes) != 1: 
        if morphemes[i] not in in_m:
          in_m[morphemes[i]] = 1/(len(self.stems))
        else:
          in_m[morphemes[i]] += 1/(len(self.stems))
      while i < (len(morphemes) - 1): #while the index is still in the bound of the word
        b = morphemes[i]
        n = morphemes[i+1]
        if n != 0: #in the cases where we have two biagrams following each other, (the initial biagram is not the last biagram of the word)
          if b in sound_prob: #if the first biagram is in the database of biagrams
            if n in sound_prob: #if the second one is in the database as well
              bInd = list(sound_prob).index(n) #find the index of the second biagram
              sound_prob[b][bInd] = int(sound_prob[b][bInd]) + 1 #and manipulate the probability value in the row of b that has the index of the following biagram
            else:
              probs = len(sound_prob) * "0" #if the second biagram does not exist in the dataset, we need to add it to the matrix and it needs a value row full of 0's (because its frequency with other sounds is 0)
              sound_prob[n] = list(probs) 
              for x in sound_prob: sound_prob[x].append(0) #every row of the matrix should be appended with 0 because the new sound that is added is also added as a new column that has never occured with the other sounds before
              sound_prob[b][-1] = 1 #the last entry of the probability of b should be altered to 1 because the new added sound appeared after that so +1 occurence
          else:
            probs = len(sound_prob) * "0" #if b does not exist in the dataset we should add it
            sound_prob[b] = list(probs)
            for x in sound_prob: sound_prob[x].append(0)
            if n in sound_prob:
              bInd = list(sound_prob).index(n)
              sound_prob[b][bInd] = int(sound_prob[b][bInd]) + 1
            else:
              sound_prob[n] = list(probs)
              sound_prob[n].append(0)
              for x in sound_prob: sound_prob[x].append(0)
              sound_prob[b][-1] = 1
        i += 1
    df = pandas.DataFrame.from_dict(sound_prob, orient="index", columns=sound_prob.keys()) #display the matrix
    df = df.applymap(int) #make every entry of the matrix an integer
    df = df/len(sound_prob.keys()) #calculate the probabilities each occurence should be divided with all occurences
    return df, sound_prob, in_m

  """  with pandas.option_context('display.max_rows', None,
                          'display.max_columns', None,
                          'display.precision', 3,
                          ): print(df)"""

"  with pandas.option_context('display.max_rows', None,\n                        'display.max_columns', None,\n                        'display.precision', 3,\n                        ): print(df)"

In [9]:
  def suffixation (self, p_list): #ascribe suffixes to the pseudostems to make them morphologically recognizable 
    global p_words
    p_words = []
    suffixes = ['о-таки', 'о-то','но','цька','ий','ик','ник','івник','льник','иво','аль','ень','ець','ість','тель','иця','иня','ння','іння','ання','яння','ення','иння','еня','ечок','ечка','ечко','ичок','ичка','енко','енько','исько','ище','івка','овка','ок','ир','ист','изм','ір','іст','ізм','яти','ати','іти']
    ending = ['б', 'в', 'г', 'ґ', 'д', 'ж', 'з', 'к', 'л', 'м', 'н', 'п', 'р', 'с', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ']
    for instance in p_list: 
        if (list(filter(instance.endswith, ending))): #if the ending of the pseudostem matches any ending of the 'ending' list, it gets a random suffix from the 'suffixes' list     
          norm = instance + random.choice(suffixes)
          if norm not in p_words: #pseudowords list gets appended with the unique pseudoword
            p_words.append(norm)
    return p_words

In [10]:
  def clean (self, ps_list): #wipe out repeating syllables, vowels or consonants (normalize them to make more natural for ukrainian)
    ps_list = re.sub(r'(.+?)\1+', r'\1', str(ps_list))   
    ps_list = literal_eval(str(ps_list)) #as the list needs to be converted into a string in the previous line, it gets converted back into a list
    return ps_list

In [11]:
  def categorize (self, dataset): #classify pseudowords by their functions within the sentence, based on their morphological endings
    uk_pos = {"SUBJECT": [],
              "PREDICATE": [],
              "ATTRIBUTE":[],
              "OBJECT": [],
              "ADVERBIAL MODIFIER": []}
    
    #suffix lists, by which functions within the sentence are implied
    subj_suff = ['ик','ник','івник','льник','иво','аль','ень','ець','ість','тель','иця','иня','ння','іння','ання','яння','ення','иння','еня','ечок','ечка','ечко','ичок','ичка','енко','исько','ище','івка','овка','ок','ир','ист','изм','ір','іст','ізм']
    pred_suff = ['ти']
    attr_suff = ['ий']
    mod_suff = ['-таки', '-то', 'но', 'ацька' ]

    pr = []
    attr = []
    obj = []

    for word in dataset: #iterate though the dataset
        if any(word.endswith(s) for s in subj_suff): #iterate through the suffix list
              uk_pos["SUBJECT"].append(word) #if there's a match, add pseudowords in the nominative case to the dictionary -- subjects
              obj.append(word) #if there's a match, add pseudowords in the nominative case to the specific list to later be inflected -- objects 
        elif any(word.endswith(mod) for mod in mod_suff): 
              uk_pos["ADVERBIAL MODIFIER"].append(word) #if there's a match, add pseudowords in the nominative case to the dictionary -- adverbial modifiers
        elif any(word.endswith(p) for p in pred_suff): 
              pr.append(word) #if there's a match, add pseudowords in the nominative case to the specific list to later be inflected -- predicates
        elif any(word.endswith(at) for at in attr_suff):
              attr.append(word) #if there's a match, add pseudowords in the nominative case to the specific list to later be inflected -- attributes

    def infl_dict(infl_suff, inf_suff, lst, dict_key): #function that inflects predicates and attributes (for accusative), and adds them to the dictionary
      for st in lst: #iterate though the list
        for af in infl_suff: #iterate through the nominative suffix list
          ret = re.sub(inf_suff, af, st) #substitute the ending in the nominative case with the ending in the accusative
          uk_pos[dict_key].append(ret) 

    infl_dict(infl_suff=['в','ла', 'ло'], inf_suff = 'ти', lst=pr, dict_key="PREDICATE")
    infl_dict(infl_suff=['ої','ого'], inf_suff = 'ий', lst=attr, dict_key="ATTRIBUTE")

    def infl_dict_obj (gendered_suffix, inflection): #function that inflects objects (for accusative) based on the inflection rules for gendered nouns (more condition neccessary for objects)
        for ob in obj: #iterate though the list
          if any(ob.endswith(s) for s in gendered_suffix): #iterate through the gendered nominative suffix list
            if ob[-1] in ['м', 'р', 'к', 'т']: #nouns ending with suffixes from the list, and with these letters specifically, get another letter added
              ob = ob + inflection
            elif ob[-1] in ['ь']: #other nouns that match change the last letter 
              ob = ob[:-1] + inflection
            elif ob[-1] in ['я']:
              ob = ob[:-1] + inflection
            elif ob[-1] in ['а']:
              ob = ob[:-1] + inflection
            elif ob[-1] in ['о', 'е']:
              ob = ob[:-1] + inflection
            uk_pos["OBJECT"].append(ob)

    infl_dict_obj(gendered_suffix=['ик','ник','івник','льник','ок','ир','ист','изм', 'ір', 'іст', 'ізм'], inflection = 'а') #masculine nouns that end in a hard consonant
    infl_dict_obj(gendered_suffix= ['аль', 'ень' 'тель', 'ець'], inflection = 'я') #masculine nouns that end in a soft consonant
    infl_dict_obj(gendered_suffix=['иця','иня','ння','іння','ання','яння','иння'], inflection = 'і') #feminine nouns that end in a soft consonant
    infl_dict_obj(gendered_suffix=['ичка', 'івка', 'овка'], inflection = 'и') #feminine nouns that end in a hard consonant
    infl_dict_obj(gendered_suffix=['иво', 'ечко', 'енко', 'исько','ище'], inflection = 'а') #neutral nouns
    return uk_pos

In [12]:
  def sent_generator(self, dic): #generate sentences with the dictionary values, based on the dictionary keys
    subj = random.choice(list(dic["SUBJECT"])) #pick a random value
    pred = random.choice(list(dic["PREDICATE"]))
    attr = random.choice(list(dic["ATTRIBUTE"]))
    obje = random.choice(list(dic["OBJECT"]))
    adve = random.choice(list(dic["ADVERBIAL MODIFIER"]))

    if re.search('[ое]$', subj): #coordinate subject and predicate by gender
        while not re.search('ло$', pred): #if a chosen subject matches a certain pattern, a predicate must match a specific pattern too -- if it doesn't, a different value is being chosen until the conditions are satisfied.
          pred = random.choice(list(dic["PREDICATE"]))     
    elif re.search('[аяь]$', subj):
        while not re.search('ла$', pred):
          pred = random.choice(list(dic["PREDICATE"]))
    elif re.search('[кмстр]$', subj):
        while not re.search('в$', pred):
          pred = random.choice(list(dic["PREDICATE"]))
      
    if re.search('[иі]$', obje): #coordinate attribute and object by gender
        while not re.search('ої$', attr): #if a chosen object matches a certain pattern, an attribute must match a specific pattern too -- if it doesn't, a different value is being chosen until the conditions are satisfied.
          attr = random.choice(list(dic["ATTRIBUTE"]))
    elif re.search('[ая]$', obje): 
        while not re.search('ого$', attr):
          attr = random.choice(list(dic["ATTRIBUTE"]))     

    sent_str = random.choice(["sent_str1", "sent_str2",  "sent_str3"]) #create random sentences using a randomly chosen structure, natural to the ukrainian syntax
    if sent_str == "sent_str1":
      sent = subj + " " + pred + " " + attr + " " + obje + " " + adve + "."
    elif sent_str == "sent_str2":
      sent = adve + " " + subj + " " + pred + " " + attr + " " + obje + "."
    else: 
      sent = attr + " " + obje + " " + pred + " " + subj + " " + adve + "."
    sent = sent.capitalize()
    return sent

In [13]:
  def run(self):
      df_prob, dict_prob, in_morph = self.probabilities()
      pre_p_words = [] 
      j=0
      while j < self.n_words: #create pseudostems
        pword = []
        i = 0
        plen = 4 
        while i < plen:
          if pword == []:
            sound = random.choices(list(in_morph.keys()), in_morph.values())[0] #start with a morpheme that is most likely to be in the beginning of the word 
            i += len(sound)
          else:
            sound = random.choices(list(dict_prob.keys()), list(df_prob[pword[-1]]))[0] #proceed with other morphemes until the length requirement is satisfied
            i += len(sound)
          pword.append(sound)
          pword = "".join(pword)
        pre_p_words.append(pword)
        p_words = self.suffixation(pre_p_words) #run function to suffixate the stems
        p_words = self.clean(p_words) #run function to normalize the pseudowords 
        j += 1

      word_categories = self.categorize(p_words) #run function to sort the parts of the sentences into the dictionary
      p_sentences = []
      for i in range(self.n_sent):
        p_sentences.append(self.sent_generator(word_categories)) #run function to create sentences
      print(p_sentences)
      return p_sentences

In [21]:
s = Pseudoword_genUK("uk_UA.csv")

AttributeError: ignored

In [20]:
s.run()

AttributeError: ignored